# Chapter 13
## 13.1 Abundance of Digitized Text
인터넷에 멀티미디어 정보가 넘쳐나지만 여전히 텍스트 프로세싱은 컴퓨터의 중요한 기능 중 하나로 남아있다. 이번 챕터에서는 큰 규모의 텍스트 데이터 셋을 효율적으로 분석하고 다룰 수 있는 핵심적인 알고리즘을 알아볼 것이다. 처음으로는 효율적이지 않지만 다양한 용도로 사용될 수 있는 **brute-force method**, 그 다음으로는 **다이나믹 프로그래밍(dynamic programming)**이라 알려진 알고리즘 테크닉을 다룰 것이다. 다이나믹 프로그래밍을 이용하면 exponential time이 필요한 문제를 polynomial time 안에 풀 수 있게 된다. 또, 텍스트 데이터 셋의 용량은 매우 큰 경우가 많기 때문에 이를 압축하기 위한 방법을 알아보기도 할 것이다. 이 때 우리는 어려운 문제에 대한 대략적인 답을 찾는 데 도움이 되는 **greedy method**를 이용할 것이다.

### 13.1.1 Notations for Strings and the Python str Class
앞으로의 알고리즘 설명을 위해 문자열 안의 문자들은 이미 알려진 **알파벳(alphabet)**으로만 이루어져 있다고 가정하고, 이 알파벳을 $\Sigma$로 쓸 것이다. 이 알파벳은 ASCII나 Unicode 문자의 부분집합일 수도 있지만 그보다 더 큰 일반적인 의미를 가질 수도 있다. 알파벳의 사이즈는 $|\Sigma|$로 고정되어 있지만, 파이썬이 유니코드 알파벳을 다루는 것처럼 이 크기는 수백만에 달할 정도로 커져서 nontrivial 할 수도 있다.

## 13.2 Pattern-Matching Algorithms
이번 섹션에서는 3개의 패턴-매칭 알고리즘을 살펴보자. 간단히 하기 위해 함수의 문법을 파이썬 `str` 클래스의 `find` 메소드와 일치시켰다.

### 13.2.1 Brute Force
**브루트 포스(brute-force)** 알고리즘 디자인 패턴은 찾고 싶은 것이 있거나, 최적화하고 싶은 함수가 있을 때 이용할 수 있는 효과적인 테크닉이다. 이 테크닉에서는 가능한 입력에 대해 모두 시도를 해보고 그 중에서 가장 결과가 좋은 입력을 선택하게 된다. 이제 이 브루트 포스 패턴 매칭 알고리즘을 구현해보자.

In [1]:
def find_brute(T, P):
    """Return the lowest index of T at which substring P begins (or else -1)."""
    n, m = len(T), len(P)
    for i in range(n-m+1):
        k = 0
        while k < m and T[i + k] == P[k]:
            k += 1
        if k == m:
            return i
    return -1

### Performance
brute-force 패턴 매칭 알고리즘의 효율성 분석은 굉장히 쉽다. 위의 코드를 보면 **for** 루프가 최대 $n-m+1$번 실행되고, **while** 루프가 최대 **m**번 실행되는 것을 볼 수 있다. 따라서 brute-force 메소드의 worst-case 작동 시간은 $O(nm)$이다.

## 13.2.2 The Boyer-Moore Algorithm
위에서는 패턴 $P$를 찾기 위해서 $T$의 모든 문자를 조사했지만, 사실 그럴 필요는 없다. **Boyer-Moore** 패턴 매칭 알고리즘은 $P$를 찾는 과정에서 $T$의 많은 부분을 생략한다. 이번 섹션에서는 Boyer-Moore 알고리즘의 간략화된 버전을 소개한다. Boyer-Moore 알고리즘의 핵심적인 아이디어는 다음의 두 휴리스틱에 기초한다:

**Looking-Glass Heuristic:** $T$ 안에서 $P$를 찾을 때 $P$의 뒤에서부터 비교를 실시한다.

**Character-Jump Heuristic:** $T$ 안에서 $P$를 찾을 때 $T[i]=c$와 $P[k]$ 사이에 미스매치가 발생한 경우, $c$가 $P$안에 포함되어 있지 않은 경우 이 $c$는 $P$를 찾는 데 전혀 도움이 되지 않으므로 $P$를 완전히 $T[i]$ 너머로 이동시킨다. 그렇지 않고 $c$가 $P$안에 있다면 $P$ 안의 $c$가 $T[i]$와 일치할 때까지 $P$를 이동시킨다.

이 두 휴리스틱은 독립적이지 않은데, Looking-Glass 휴리스틱과 함께 다른 휴리스틱을 이용하면 $P$를 찾기 위해 $T$의 모든 문자를 찾아야 할 필요가 없어지기 때문이다. 아래의 그림은 이 휴리스틱들을 적용하면 어떻게 탐색이 이루어지는지를 그림으로 나타낸 예제이다.

<img width="640" alt="figure-13.2" src="https://user-images.githubusercontent.com/20944657/37551757-463cc44c-29ea-11e8-92bc-4afa4f19d012.png">

이제 두 문자간의 불일치가 일어나는 경우의 인덱스를 자세히 살펴보자. 만약 미스매치가 일어난 $T$의 문자가 $P$에 없다면 전체 패턴이 그 위치를 넘어 이동하게 될 것이고, 만약 패턴 내에 그 문자가 있다면 두 경우를 살펴봐야 한다. 첫째는 패턴 내에서 그 문자의 마지막 위치(last occurrence)가 패턴 내에서 불일치가 일어난 문자의 앞에 나타나는 경우이고, 둘째는 불일치가 일어난 문자의 뒤에 일어나는 경우다. 이는 아래의 그림을 보면 잘 이해할 수 있다. 아래의 그림에서 $i$는 $T$에서 미스매치가 일어난 문자의 인덱스, $k$는 패턴 내에서 미스매치가 일어난 문자의 인덱스, $j$는 패턴 내에서 $T[i]$의 마지막 위치 인덱스를 의미한다.

<img width="640" alt="figure-13.3a" src="https://user-images.githubusercontent.com/20944657/37551778-d7564a66-29ea-11e8-8aaf-bc59f69141b7.png">
<img width="640" alt="figure-13.3b" src="https://user-images.githubusercontent.com/20944657/37551779-d78248c8-29ea-11e8-87d9-e5a7c764f753.png">

첫번째 케이스가 발생한 경우, 즉 $j<k$인 경우에 우리는 패턴을 $k-j$만큼 이동시키고, 이 경우 $i$는 $m-(j+1)$만큼 이동한다. 두번째 케이스가 발생한 경우, $j>k$인 경우엔 패턴을 한 칸만 이동시키면 되고, 이 경우 $i$는 $m-k$만큼 이동한다. 그런데, 두번째 케이스에서도 한 칸만 이동하는게 아니라 패턴 내에서 $T[i]$를 찾아 두 문자가 일치하게끔 이동하면 되지 않느냐 하는 의문을 가질 수도 있다. 그러나 여기서 그렇게 하지 않는 이유는 또 다른 occurrence를 찾기 위해 추가적인 시간을 쓰지 않게 하기 위함이다.

Boyer-Moore 알고리즘의 효율성은 패턴 내에서 불일치가 일어난 문자가 존재하는지를 빠르게 결정할 수 있게 해주는 룩업 테이블의 존재에 의존한다. 특히 우리는 함수 $last(c)$를 다음과 같이 정의한다.

- 만약 $c$가 $P$에 있다면 $last(c)$는 $P$ 내에서 $c$의 마지막(가장 오른쪽) 인덱스가 된다. 만약 $P$에 없다면 $last(c) = -1$로 정의한다.

알파벳이 고정된 유한 크기라고 가정하고 문자들이 배열의 인덱스로 변환될 수 있다고 하면 `last` 함수는 `last(c)` 접근에 worst-case $O(1)$ time이 걸리는 룩업 테이블로 쉽게 구현할 수 있다. 그러나 이 테이블은 알파벳의 크기 만큼의 사이즈를 가질 것이므로(패턴의 사이즈가 아님), 전체 테이블을 초기화하는 데 시간이 필요할 것이다. 

우리는 위와 같이 룩업 테이블을 구현하지 않고, 패턴 내에 실제로 등장하는 문자로만 구성된 해쉬 테이블을 이용해서 `last` 함수를 구현할 것이다. 그러면 이러한 접근의 공간 복잡도는 패턴에 등장하는 각각의 문자 기호들의 수에 비례하게 될 것이고, $O(m)$이 될 것이고, 문제에 관계 없이 예상 lookup time이 결정될 것이다.

In [2]:
def find_boyer_moore(T,P):
    """Return the lowest index of T at which substring P begins (or else -1)."""
    n, m = len(T), len(P)                         # introduce convenient notations
    if m == 0: return 0                           # trivial search for empty string
    last = {}                                     # build 'last' dictionary
    for k in range(m):
        last[P[k]] = k                            # later occurrence overwrites
    # align end of pattern at index m-1 of text
    i = m-1                                       # an index into T
    k = m-1                                       # an index into P
    while i < n:
        if T[i] == P[k]:                          # a matching character
            if k == 0:
                return i                          # pattern begins at index i of text
            else:
                i -= 1                            # examine previous character
                k -= 1                            # of both T and P
        else:
            j = last.get(T[i], -1)                # last(T[i]) is -1 if not found
            i += m - min(k,j+1)                   # case analysis for jump step
            k = m -1                              # restart at end of pattern
    return -1

Boyer-Moere 알고리즘이 실패하지 않을 수 있는 이유는 아래의 그림과 같이 shift가 이루어질 때 가능한 매치를 "뛰어넘지" 않기 때문이다. 

<img width="640" alt="figure-13.4" src="https://user-images.githubusercontent.com/20944657/37551907-9e895cc0-29ed-11e8-99c8-3b32aad1d40b.png">

### Performance
만약 전통적인 룩업 테이블을 이용한다면 Boyer-Moore 알고리즘의 worst-case time은 $O(nm + |\Sigma|)$가 된다. 그 이유는 `last` 함수의 계산이 $O(m+|\Sigma|)$가 걸리고 실제 패턴의 탐색은 $O(nm)$ 시간이 걸리기 때문이다(해쉬 테이블을 이용하면 $|\Sigma|$에 대한 의존이 사라진다). worst-case의 예시로는 아래의 경우가 있는데, 이는 영어에서는 실제로 일어날 확률이 거의 없기 때문에 Boyer-Moore 알고리즘은 텍스트의 많은 부분을 건너뛰며 탐색하게 된다. 실증 연구에 따르면 5개의 문자로 이루어진 패턴을 탐색할 때 각 문자마다 이루어지는 평균 비교 회수가 0.24라 한다.

우리가 다룬 Boyer-Moore 알고리즘은 간략화된 버전이고, 원래의 알고리즘은 $O(n+m+|\Sigma|)$ 시간이 걸린다. 이 원래의 알고리즘은 다음에서 다룰 Knuth-Morris-Pratt 패턴 매칭 알고리즘의 핵심적인 아이디어에 의존하고 있다.

## 13.2.3 The Knuth-Morris-Pratt Algorithm
brute-force와 Boyer-Moore 패턴 매칭 알고리즘의 worst-case 퍼포먼스를 보다 보면 어디서 비효율성이 나타나는 지 찾을 수 있는데, 위의 두 알고리즘에서는 미스매치를 찾게 되면 이전의 성공적인 매치에서 얻은 정보는 다 무시하고 새로 비교를 시작하게 되어 정보의 낭비가 발생한다. Knuth-Morris-Pratt("KMP") 알고리즘은 이러한 정보의 낭비를 피하고 $O(n+m)$ 시간 복잡도를 달성하는 알고리즘이다. 즉, 최악의 경우에 이 알고리즘은 텍스트와 패턴의 문자를 한번씩만 조사하는 것으로 끝난다. KMP 알고리즘의 핵심적인 아이디어는 패턴의 부분들 사이의 겹치는 부분을 미리 계산하고, 미스매치가 일어난 경우 탐색을 다시 실시하기 전에 최대한 얼마나 패턴을 이동시켜야 할 지 바로 알아내서 효율적으로 탐색을 실시하는 것이다.

<img width="640" alt="figure-13.5" src="https://user-images.githubusercontent.com/20944657/37553073-df8ffb6e-2a03-11e8-84ec-dbb1bad996c2.png">

위의 그림에서 만약 화살표가 위치하는 장소에서 미스매치가 발생한다면 패턴은 prefix `ama`가 텍스트와 일치하는지 다시 확인할 필요 없이 두번째 패턴처럼 이동하는 것이 가능하다. 만약 화살표가 가리키는 장소에 있는 문자가 `l`도 아니라면 패턴의 다음 위치는 두번 들어있는 똑같은 문자 `a`를 이용해서 정해진다. 아직 모호하다면 아래의 실패 함수(failure function)와 구현을 보면 이해가 될 것이다.

### The Failure Function
KMP 알고리즘을 구현하기 위해 우리는 **실패 함수(failure function)** $f$를 미리 계산해서, 매칭이 실패한 경우 $P$를 얼마나 이동시켜야 하는지 미리 알아낼 것이다. 실패 함수 $f(k)$는 suffix $P[1:k+1]$ 중에서 가장 긴 $P$의 prefix 길이로 정의된다(여기서 $P[0]$은 포함하지 않는데, 적어도 한 칸은 이동해야 하기 때문이다). 직관적으로, 만약 $P[k+1]$에서 미스매치가 발생한 경우 $f(k)$는 지금까지 매칭한 문자 중에서 얼마나 많은 문자가 재탐색에 이용될 수 있는지를 알려주는 함수이다. 아래는 위의 그림에서의 $f(k)$ 값을 계산해서 표로 나타낸 것이다.

<img width="640" alt="example-13.2" src="https://user-images.githubusercontent.com/20944657/37553158-ac6bf70e-2a05-11e8-9262-6675008f8403.png">

### Implementation
이제 KMP 패턴 매칭 알고리즘을 구현해보자. 아래의 구현에서는 `compute_kmp_fail` 실패 함수를 효율적으로 계산하는 `compute_kmp_fail` 함수를 이용할 것이다. KMP 알고리즘의 메인 파트는 **while** 루프 안에 있는데, 이 안에서 $T$의 인덱스 $j$와 $P$의 인덱스 $k$ 사이의 문자 비교가 이루어진다.

In [3]:
def find_kmp(T, P):
    """Return the lowest index of T at which substring P begins (or else -1)."""
    n, m = len(T), len(P)
    if m == 0: return 0
    fail = compute_kmp_fail(P)
    j = 0
    k = 0
    while j < n:
        if T[j] == P[k]:
            if k == m - 1:
                return j - m + 1
            j += 1
            k += 1
        elif k > 0:
            k = fail[k-1]
        else:
            j += 1
    return -1

이제 `compute_kmp_fail`을 구현해보자. 여기서는 KMP 알고리즘을 이용해서 자기 자신을 패턴과 비교하는데, 이는 앞에서 봤던 "부트스트래핑(bootstrapping)" 방법이다. KMP 알고리즘을 구현하기 위해 사용하는 유틸리티 메소드가 KMP 알고리즘을 이용해 구현되기 때문이다. 이 때 주목해야 할 부분은 실패 함수의 이전 값들을 이용해서 새로운 값들을 효율적으로 계산하고 있다는 점이다.

In [4]:
def compute_kmp_fail(P):
    """Utility that computes and returns KMP 'fail' list."""
    m = len(P)
    fail = [0] * m              # by default, presume overlap of 0 everywhere
    j = 1
    k = 0
    while j < m:
        if P[j] == P[k]:        # compute f(j) during this pass, if nonzero
            fail[j] = k + 1     # k + 1 characters match thus far
            j += 1
            k += 1
        elif k > 0:             # k follows a matching prefix
            k = fail[k-1]
        else:                   # no match found starting at j
            j += 1
    return fail

### Performance
실패 함수의 실행 시간을 제외하면 KMP 알고리즘의 실행 시간은 **while** 루프의 iteration 수에 비례한다. 분석의 편의를 위해 $s = j - k$라 하자. 직관적으로 $s$는 패턴 $P$가 이동한 정도를 나타낸다. 알고리즘 내내 $s \leq n$이 성립한다. 이제 루프의 각 iteration마다 다음의 세 케이스 중 하나가 반드시 발생한다:

- $T[j] = P[k]$이면 $j$,$k$가 1씩 증가해서 $s$가 변화하지 않는다.
- $T[j] \neq P[k]$이고 $k \gt 0$이면 $j$는 변화하지 않고 $s$는 최소 1 증가한다. 이 경우 $s$는 $j-k$에서 $j-f(k-1)$으로 변화하는데, 이는 $k-f(k-1) \gt 0$이 더해진 것과 같다.
- $T[j] \neq P[k]$이고 $k = 0$이면 $j$는 1 증가하고 $s$도 1 증가한다.

따라서 루프의 iteration마다 $j$나 $s$가 최소 1씩 증가하기 때문에 **while** 루프의 최대 iteration 수는 $2n$이 된다. 이제 실패 함수를 살펴보면 되는데, KMP 알고리즘과 유사한 방식으로 실패 함수의 시간 복잡도가 $O(m)$임은 쉽게 알 수 있다. 따라서,

**Proposition 13.3:** Knuth-Morris-Pratt 알고리즘은 $O(n+m)$ 시간 안에 길이가 $n$인 텍스트에서 길이가 $m$인 패턴에 대한 탐색을 실행할 수 있다.

KMP 알고리즘의 정확성은 실패 함수의 정의에 의존한다. 실패 함수에 의해 생략되는 비교들은 실제로 불필요한 것들인데, 똑같은 비교를 다시 실행하는 것이기 때문이다. 

<img width="640" alt="figure-13.6" src="https://user-images.githubusercontent.com/20944657/37553438-7ca39012-2a0b-11e8-97d5-ea2a941ec736.png">

## 13.3 Dynamic Programming
이번 섹션에서는 **동적 프로그래밍(dynamic programming)** 알고리즘 디자인 테크닉을 다룬다. 이 테크닉은 다양한 문제들에 적용될 수 있다는 점에서 분할-정복(divide-and-conquer) 알고리즘과 유사하다. 동적 프로그래밍은 지수 시간을 필요로 하는 문제를 다항 시간이 걸리는 알고리즘으로 해결하는 데에 이용된다. 동적 프로그래밍의 장점은 이러한 과정에서 이용되는 다항 시간 알고리즘이 몇 줄의 코드만으로도 구현할 수 있을 정도로 굉장히 간단하다는 것이다.

## 13.3.1 Matrix Chain-Product
동적 프로그래밍 테크닉의 일반적인 요소를 설명하기 전에 고전적이면서도 구체적인 예제부터 살펴보자. 이 예제에서 우리는 $n$개의 2차원 행렬을 이용해서 다음의 곱을 계산하고자 한다.

$$A = A_{0} \cdot A_{1} \cdot A_{2} \cdots A_{n-1}$$

이 때 $A_{i}$는 $d_{i} \times d_{i+1}$ 행렬이다. 표준적인 행렬 곱셈 알고리즘에서는 $d \times e$ 행렬 $B$와 $e \times f$ 행렬 $C$를 곱한 $A$를 구하기 위해 다음의 식을 계산한다.

$$A[i][j] = \sum_{k=0}^{e-1}B[i][k] \cdot C[k][j]$$ 

이 정의에 따르면 행렬 곱셈에서 결합(associative) 법칙이 성립함을 알 수 있다. 즉, $B \cdot (C \cdot D) = (B \cdot C) \cdot D$가 성립한다. 이를 이용하면 위에서 $A$를 계산하는 과정에서 오른쪽의 행렬들을 마음대로 괄호로 묶어도 같은 결과를 얻을 수 있음을 안다. 그런데 중요한 점은 괄호를 어떻게 묶느냐에 따라 계산의 수가 달라진다는 것이다. 다음의 예제를 보자.

**Example 13.4:** $B$를 $2 \times 10$ 행렬, $C$를 $10 \times 50$ 행렬, $D$를 $50 \times 20$ 행렬이라 하자. $B \cdot (C \cdot D)$를 계산하기 위해서는 $2 \cdot 10 \cdot 20 + 10 \cdot 50 \cdot 20 = 10400$번의 곱셈을 해야하지만 $(B \cdot C) \cdot D$를 계산하기 위해서는 $2 \cdot 10 \cdot 50 + 2 \cdot 50 \cdot 20 = 3000$번의 곱셈만 하면 충분하다.

**matrix chain-product** 문제는 총 스칼라 곱셈 수를 최소화하기 위해 $A$를 정의하는 식에서 괄호를 어떻게 쳐야할 지를 결정하는 문제이다. 위의 예제에서 알 수 있듯이 괄호를 묶는 방식에 따른 차이가 엄청나기 때문에 좋은 해답을 찾으면 계산 속도를 크게 늘릴 수 있을 것이다.

### Defining Subproblems
matrix chain-product 문제를 풀기 위한 한가지 방법은 $A$를 괄호로 묶는 모든 방법을 나열하고 각각의 경우를 계산해서 최선의 방법을 찾는 것이다. 불행히도 이는 $n$개의 잎을 갖는 이진 트리의 모든 경우의 수와 같고, 그 수는 $n$의 지수 꼴이 된다. 따라서 brute-force 알고리즘은 exponential time을 갖는다.

그러나 matrix chain-product 문제를 잘 관찰해보면 brute-force 알고리즘의 시간을 줄일 수 있는 방법을 찾을 수 있다. 첫째는 이 문제가 여러개의 **하위 문제(subproblem)**들로 분리될 수 있다는 것이다. 예를 들어, 이 문제는 $A_{i} \cdot A_{i+1} \cdots A_{j}$와 같이 식의 일부분에 대해서 최적의 괄호를 찾는 문제들로 나눌 수 있다. $N_{i,j}$를 이 하위 식을 계산하기 위해 필요한 최소 곱셈의 수로 정의하자. 그러면 원래 문제는 $N_{0,n-1}$을 찾는 문제로 표현할 수 있다. 이러한 관찰이 중요하긴 하지만 동적 프로그래밍 테크닉을 이용하기 위해서는 또 다른 관찰이 필요하다.

### Characterizing Optimal Solutions
matrix chain-product 문제를 위한 또 다른 중요한 관찰은 특정 하부 문제에 대한 최적 솔루션을 그 하부문제의 하부 문제의 최적 솔루션들로 나타내는 것이 가능하다는 것이다. 우리는 이러한 성질을 **하부문제 최적성(subproblem optimality)** 조건이라 부른다.

matrix chain-product 문제의 경우, 하위 표현(subexpression)을 어떻게 괄호 치는가에 관계없이 결국 실행해야 할 최종적인 행렬 곱셈이 존재한다. 즉, 하위 표현식 $A_{i} \cdot A_{i+1} \cdots A_{j}$에 괄호를 치게 되면 어떤 $k \in {i,i+1,...,j-1}$에 대해 $(A_{i}\cdots A_{k})\cdot(A_{k+1}\cdots A_{j})$ 꼴이 될 것이다. 만약 $k$를 적절하게 선택했다면 곱셈 $(A_{1}\cdots A_{k})$와 $(A_{k+1}\cdots A_{j})$는 optimally solve될 것이다. 만약 그렇지 않다면 이 하부 문제 중 하나가 최적이 아님에도 불구하고 전체 곱셈이 최적이 되는 전역 최적 해가 존재한다는 말인데, 이러한 경우 최적이 아닌 하부 문제의 해를 최적 해로 바꿔서 총 곱셈의 수를 줄이는 것이 가능하므로 그러한 전역 최적해가 존재한다는 것은 불가능하다. 이러한 관찰로부터 알 수 있는 사실은 $N_{i,j}$를 구하는 최적화 문제를 여러 하부 문제로 표현하는 것이 가능하다는 것이다. 우리는 최종적인 곱셈이 이루어지는 $k$를 정하기만 하면 $N_{i,j}$를 계산할 수 있고, 그러한 $k$ 중에서 최적 $k$를 찾기만 하면 된다.

### Designing a Dynamic Programming Algorithm
위의 논의를 종합하면, 우리는 하부문제의 최적 해인 $N_{i,j}$를 다음과 같이 정의할 수 있다.

$$N_{i,j} = \min_{i \leq k \lt j} \{N_{i,k} + N_{k+1,j} + d_{i}d_{k+1}d_{j+1}\}$$

위에서 $d_{i}d_{k+1}d_{j+1}$ 항은 $(A_{i}\cdots A_{k})$가 $d_{i} \times d_{k+1}$ 행렬이고, $A_{k+1} \cdots A_{j}$가 $d_{k+1} \times d_{j+1}$ 행렬이라는 사실에서 도출된 것이다. 행렬이 하나만 있는 경우는 곱셈이 필요하지 않으므로 $N_{i,i} = 0$이다. 위의 식으로부터 우리는 $N_{i,j}$가 최종 곱셈의 수를 계산하기 위한 모든 경우의 수를 고려할 때 얻을 수 있는 최적의 곱셈 수임을 알 수 있다.

그런데 위의 문제에서 주목해야 할 점은 **하부 문제의 공유(sharing of subprolems)**가 일어나고 있어서 분할-정복 테크닉에서와 같이 완전히 독립적인 하부문제들로 문제를 분할하는 것이 불가능하다는 점이다. 그럼에도 불구하고 우리는 위 식을 이용해서 $N_{i,j}$를 bottom-up으로 계산하면서 intermediate solution들을 $N_{i,j}$ 값의 테이블에 저장하는 방식을 통해 $N_{i,j}$를 계산하는 효율적인 알고리즘을 도출할 수 있다. 이를 위해서는 우선 $i = 0,1,...,n-1$에 대해 $N_{i,i} = 0$으로 둔다. 그리고 나서는 $N_{i,j}$의 식을 이용해서 $N_{i,i+1}$ 식을 구한다. 그러면 이제 $N_{i,i+1}$을 이용해서 $N_{i,i+2}$를 구할 수 있다. 이러한 과정을 반복해서 우리가 찾던 값인 $N_{0,n-1}$을 찾을 수 있다. 이러한 **동적 프로그래밍(dynamic programming)**의 파이썬 구현은 아래와 같다.

In [6]:
def matrix_chain(d):
    """d is a list of n+1 numbers such that size of kth matrix is d[k] by d[k+1].
    
    Return an n-by-n table such that N[i][j] represents the minimum number of
    multiplications needed to compute the product of Ai through Aj inclusive.
    """
    n = len(d) - 1                        # number of matrices
    N = [[0] * n for i in range(n)]       # initialize n-by-n result to zero
    for b in range(1, n):                 # number of products in subchain
        for i in range(n-b):              # start of subchain
            j = i + b                     # end of subchain
            N[i][j] = min(N[i][k] + N[k+1][j] + d[i]*d[k+1]*d[j+1] for k in range(i,j))
    return N

위의 방법을 이용하면 우리는 세개의 중첩 루프를 이용해서 $N_{0,n-1}$을 구할 수 있고, 각각의 루프가 최대 $n$ 시간이 걸리므로 이 알고리즘의 총 작동 시간은 $O(n^{3})$이다.

## 13.3.2 DNA and Text Sequence Alignment
유전학과 소프트 엔지니어링에서 주로 마주하게 되는 텍스트 프로세싱 문제는 두 텍스트 사이의 유사성을 판단하는 문제이다. 유전학에서는 주로 DNA 가닥에 해당하는 두 문자열의 유사성을 비교하고, 소프트웨어 공학에서는 같은 프로그램의 두 소스코드 버전을 비교해서 어떤 차이가 있는지 결정하게 된다. 사실 두 문자열의 유사성을 판단하는 것은 흔한 일이라서 Unix와 Linux는 `diff`라는 built-in 프로그램을 통해 텍스트 파일을 비교한다.

문자열 $X = x_{0}x_{1}x_{2}\cdots x_{n-1}$이 있을 때, $X$의 **subsequence**는 $x_{i_{1}}x_{i_{2}}\cdots x_{i_{k}}$, where $i_{j} \lt i_{j+1}$로 이루어진 문자열을 의미한다. 즉, subsequence는 연속적이지는 않지만 $X$의 순서를 유지하는 부분 시퀀스라고 볼 수 있다. 예를 들어 $AAAG$는 $CG\underline{A}T\underline{AA}TT\underline{G}AGA$의 subsequence이다.

우리가 앞서 언급한 DNA와 텍스트 유사도 문제는 **longest common subsequence**(LCS) 문제라고도 한다. 이 문제에서 우리는 주어진 알파벳들로 이루어진(유전학에서는 주로 $\{A,C,G,T\}$의 알파벳을 이용) 두 문자열 $X=x_{0}x_{1}x_{2}\cdots x_{n-1}$, $Y=y_{0}y_{1}y_{2}\cdots y_{m-1}$을 받아서 $X$와 $Y$의 subsequence가 되는 가장 긴 문자열 $S$를 찾게 된다. 이 LCS 문제를 푸는 한가지 방법은 $X$의 모든 subsequence을 나열하고 그 중에서 $Y$의 subsequence가 되면서도 가장 긴 문자열을 찾는 것이다. $X$의 각각의 문자는 subsequence에 포함될 수도 있고 아닐 수도 있으므로 $2^{n}$개의 subsequence가 존재한다. 또, 각각의 subsequence가 $Y$의 subsequence가 되는지 결정하기 위해서는 $O(m)$ 시간이 필요하다. 따라서 이러한 brute-force는 $O(2^{n}m)$시간이 걸리고 이는 굉장히 비효율적이다. 다행히도 우리는 **동적 프로그래밍**을 이용해서 LCS 문제를 효율적으로 풀 수 있다.

### The Components of a Dynamic Programming Solution
위에 언급했듯이 동적 프로그래밍 테크닉은 주로 무언가를 하기 위한 "가장 좋은" 방법을 찾는 **최적화** 문제에 이용된다. 우리는 문제가 다음과 같은 성질을 가질 때 동적 프로그래밍 테크닉을 이용할 수 있다:
- **Simple Subproblems:** 전역 최적 문제를 하위 문제들로 반복적으로 분해할 수 있어야 한다.
- **Subproblem Optimization:** 전역 문제의 최적 해는 하위 문제들의 최적해들로 구성된다.
- **Subproblem Overlap:** 서로 관련이 없는 하위 문제들의 최적해가 공통적인 하위 문제들을 포함한다.

당장 위의 세 성질이 이해가 되지 않더라도 아래에서 문제를 분석하는 과정을 통해 이해할 수 있을 것이다.

### Applying Dynamic Programming to the LCS Problem
LCS 문제에서는 $X$와 $Y$가 문자열이기 때문에 하위 문제를 정의하는 데에 인덱스를 이용할 수 있다. 이제 $L_{i,j}$를 $X$와 $Y$의 prefix $X[0:j]$, $Y[0:k]$의 subsequence이면서 가장 긴 문자열의 길이라 두자. 그러면 LCS의 하위 문제는 $L_{j,k}$를 찾는 것이 된다. 이러한 정의를 하게 되면 $L_{j,k}$를 하위 문제의 최적해들로 다시 쓰는 것이 가능해진다.

<img width="640" alt="figure-13.7" src="https://user-images.githubusercontent.com/20944657/37555103-a6f9c286-2a25-11e8-9137-e8b80e59aba2.png">

- $x_{j-1} = y_{k-1}$.
이 경우 $X[0:j]$와 $Y[0:k]$의 마지막 문자가 일치하고, 이 문자는 $X[0:j]$와 $Y[0:k]$의 longest common subsequence에 속하게 된다. 만약 속하지 않는다고 하자. 이제 longest common subsequence $x_{a_{1}}x_{a_{2}}...x_{a_{c}} = y_{b_{1}}y_{b_{2}}...y_{b_{c}}$를 두자. 만약 $x_{a_{c}} = x_{j-1} or y_{b_{c}} = y_{k-1}$이면 우리는 $a_{c} = j-1$ and $b_{c} = k-1$로 둬도 같은 subsequence를 얻는다. 만약 $x_{a_{c}} \neq x_{j-1} and y_{b_{c}} \neq y_{k-1}$인 경우 끝에 $x_{j-1} = y_{k-1}$을 추가해서 더 긴 common subsequence를 얻을 수 있다. 따라서 longest common subsequence는 $x_{j-1}$로 끝나게 되고, 아래의 식이 성립한다.

$$L_{j,k} = 1 + L_{j-1,k-1} \hspace{0.4cm} if \hspace{0.2cm}x_{j-1} = y_{k-1}$$

- $x_{j-1} \neq y_{k-1}$.
이 경우 $x_{j-1}$와 $y_{k-1}$을 동시에 포함하는 subsequence는 존재하지 않는다. 즉, 우리는 $x_{j-1}$으로 끝나거나, $y_{k-1}$로 끝나는 common subsequence를 가질 수도 있지만 둘 모두를 포함하는 subsequnece를 찾을 수는 없다. 따라서 아래의 식이 성립한다.

$$L_{j,k} = \max \{L_{j-1,k}, L_{j,k-1}\}\hspace{0.4cm} if \hspace{0.2cm}x_{j-1} \neq y_{k-1}$$

이 때 $Y[0:0]$와 $X[0:0]$은 빈 string이므로 $L_{j,0} = 0$, $L_{0,k} = 0$이 성립한다.

### The LCS Algorithm
위의 식을 통해 우리는 $L_{j,k}$가 subproblem optimization 성질을 만족함을 알 수 있는데, 하위 문제에 대한 longest common subsequence가 없이 문제의 longest common subsequence를 찾을 수는 없기 때문이다. 또한 subproblem overlap도 이용되고 있는데, subproblem solution인 $L_{j,k}$가 다른 여러 문제들을 푸는 데에 이용되고 있기 때문이다($L_{j+1,k}, L_{j,k+1}, L_{j+1,k+1}$).

이를 구현하는 것은 굉장히 직관적이다. 아래의 코드를 보자.

In [7]:
def LCS(X,Y):
    """Return table such that L[j][k] is length of LCS for X[0:j] and Y[0:k]."""
    n, m = len(X), len(Y)                          # introduce convenient notations
    L = [[0] * (m+1) for k in range(n+1)]          # (n+1) x (m+1) table
    for j in range(n):
        for k in range(m):
            if X[j] == Y[k]:                       # align this match
                L[j+1][k+1] = L[j][k] + 1
            else:                                  # choose to ignore one character
                L[j+1][k+1] = max(L[j][k+1], L[j+1][k])
    return L

LCS 알고리즘의 작동시간을 분석하는 것은 굉장히 쉬운 일인데, 알고리즘이 두 개의 중첩 **for** 루프로 이루어져있기 때문이다. 외부 루프에서는 $n$번의 순회가 이루어지고, 내부 루프에서는 $m$번의 순회가 이루어진다. 루프 내의 if 구문과 할당이 $O(1)$ 시간복잡도를 가지므로 알고리즘은 $O(nm)$의 시간 복잡도를 갖는다. 따라서 동적 프로그래밍 테크닉을 이용하면 brute-force의 지수 시간보다 월등히 성능을 끌어올리는 것이 가능하다.

그런데 위의 구현에서는 longest common subsequence의 길이를 구할 뿐 그 subsequence 자체를 구하진 않는다. 다행히도 LCS 함수를 이용해서 구한 $L_{j,k}$ 값의 테이블이 있다면 실제 시퀀스를 구하는 것은 굉장히 간단하다. 우리는 $L_{n,m}$의 계산을 리버스 엔지니어링해서 그 시퀀스를 구할 수 있다. 만약 $L_{j,k}$에서 $x_{j} = y_{k}$라면 그 길이는 $L_{j-1,k-1}$인 common subsequence에 $x_{j}$가 추가된 길이이므로 $x_{j}$를 시퀀스 안에 기록하자. 그러고 나서 다시 $L_{j-1, k-1}$부터 분석을 재개한다. 만약 $x_{j} \neq y_{k}$라면 $L_{j,k-1}$, $L_{j-1,k}$ 중에 더 큰 subsequence로 이동한다. 이를 $L_{j,k}=0$일 때까지 실시하면 된다. 

이러한 과정의 구현은 아래 코드와 같다. 이 함수는 $j$나 $k$를 따라 루프를 돌게 되므로 $O(n+m)$ 시간 안에 시퀀스를 복구할 수 있다.

In [8]:
def LCS_solution(X, Y, L):
    """Return the longest common substring of X and Y, given LCS table L."""
    solution = []
    j,k = len(X), len(Y)
    while L[j][k] > 0:                     # common characters remain
        if X[j-1] == Y[k-1]:
            solution.append(X[j-1])
            j -= 1
            k -= 1
        elif L[j-1][k] >= L[j][k-1]:
            j -= 1
        else:
            k -= 1
    return ''.join(reversed(solution))      # return left-to-right version

<img width="640" alt="figure-13.8" src="https://user-images.githubusercontent.com/20944657/37555392-d23d2b0a-2a29-11e8-8c07-7753f8f11899.png">

## 13.4 Text Compression and the Greedy Method
이번 섹션에서는 중요한 텍스트 프로세싱 도구인 **텍스트 압축(text compression)**을 다룬다. 이 문제에서 우리는 ASCII나 Unicode 알파벳으로 이루어진 문자열 $X$를 받아 이 문자열을 용량이 작은 이진 문자열(binary string) $Y$로 효율적으로 인코딩해야 한다. 텍스트 압축은 디지털 커뮤니케이션의 대역폭을 줄여서 텍스트 전송에 필요한 시간을 줄여야 하는 상황에 굉장히 유용하다. 또, 텍스트 압축은 규모가 큰 문서를 효율적으로 저장해서 한정된 용량의 저장 기기 안에 가능한 많은 문서를 저장할 수 있게끔 한다.

이번 섹션에서는 **허프만 코드(Huffman code)**라 알려진 텍스트 압축 방법을 알아본다. ASCII와 같은 표준 인코딩 계획에서는 문자를 인코딩하기 위해 고정된 길이의 이진 문자열을 이용한다. 유니코드 시스템은 원래 16비트 고정 길이 표현으로 고안되었지만 일반적으로는 ASCII 문자들과 같이 적은 비트만으로 표현할 수 있는 문자들의 공간 사용량을 줄이게끔 인코딩된다. 허프만 부호는 고정 길이 인코딩에 대해서는 자주 이용되는 문자에 짧은 code-word string을 이용하고, 잘 이용되지 않는 문자에 long code-word string을 이용해서 공간을 절약한다. 또한, 허프만 부호는 알파벳으로 이루어진 문자열 $X$가 주어졌을 때 특별히 최적화된 가변길이 인코딩을 이용한다. 이 최적화는 문자 **빈도(frequency)**를 이용하며, 문자 $c$가 문자열 $X$에 나타나는 수를 $f(c)$로 둔다.

문자열 $X$를 인코딩하기 위해 우리는 $X$의 각각의 문자를 가변 길이 code-word로 변환하고 $X$에 대응되는 인코딩 $Y$를 만들기 위해 이 code-word들을 합친다. 모호성(ambiguity)을 피하기 위해 우리는 인코딩 안에서 어떤 code-word가 다른 code-word의 prefix가 되지 않는다고 할 것이다. 이러한 code를 **prefix code**라고 하고, 이 경우 $Y$를 디코딩해서 $X$를 얻는 것이 굉장히 간단해진다. 이러한 제약조건에도 불구하고 가변 길이 prefix code를 이용해서 얻을 수 있는 공간 절약은 매우 크고, 특히 문자 빈도 사이에 분산이 큰 경우 더 큰 효과를 얻을 수 있다(인간이 작성한 대부분의 자연어 텍스트에서는 대부분 이러한 성질이 만족된다).

$X$에 대한 최적 가변-길이 prefix code를 만드는 허프만 알고리즘은 code를 표현하는 이진 트리 $T$를 이용한다. $T$의 각각의 엣지는 code-word의 비트를 표현하고, 왼쪽 엣지는 "0", 오른쪽 엣지는 "1"을 의미한다. 각각의 잎 $v$는 특정 문자와 연결되어 있으며, 이 문자의 code-word는 $T$의 루트부터 $v$까지의 경로에 나타나는 엣지들로 이루어진다. 각각의 잎 $v$는 **빈도(frequency)**, $f(v)$를 갖고 있으며, 이는 $X$에서 문자 $v$가 등장한 빈도를 의미한다. 또, 우리는 각각의 내부 노드 $v$에 빈도 $f(v)$를 두는데, 이는 $v$를 루트로 하는 서브트리의 모든 잎들의 빈도를 합친 것이다.

아래는 문자열 $X$ = "a fast runner need never be afraid of the dark" 를 허프만 코드로 표현한 예제이다.

<img width="640" alt="figure-13.9a" src="https://user-images.githubusercontent.com/20944657/37555566-b84c0218-2a2c-11e8-91ee-0a92363d1ad6.png">
<img width="640" alt="figure-13.9b" src="https://user-images.githubusercontent.com/20944657/37555567-b87b2f5c-2a2c-11e8-83a9-48dba3d7a1e4.png">

### The Huffman Coding Algorithm
허프만 코딩 알고리즘은 인코딩할 각각의 $d$ 문자들을 각각을 루트로 하는 single-node 이진 트리로 만드는 데에서 시작한다. 이제 알고리즘은 여러 라운드를 거치게 되는데, 각각의 라운드에서 알고리즘은 가장 작은 빈도수를 가진 두 이진 트리를 합쳐 하나의 이진 트리로 만든다. 이러한 과정은 트리가 하나 남을 때까지 반복된다. 허프만 알고리즘에서 **while** 루프의 각각의 iteration은 힙을 이용한 우선순위 큐를 통해 $O(\log d)$ 시간 안에 구현할 수 있다.또, 각각의 iteration에서는 $Q$에서 두 노드를 꺼내고 하나의 노드를 집어넣는데, 이러한 과정은 $Q$에 노드가 하나만 남을 때까지 $d-1$번 반복된다. 따라서 이 알고리즘의 시간 복잡도는 $O(n+d\log d)$가 된다.

왜 이 알고리즘의 정확도가 보장되는지를 설명하는 것은 이 책의 범위를 넘지만, 직관적으로는 간단한 아이디어에 기반하고 있다. 그 아이디어란, 임의의 최적 코드는 또 다른 최적 코드로 변환될 수 있는데, 이 최적코드에서는 두 최저빈도 문자 $a$와 $b$가 마지막 비트만 다르게 표현된다는 것이다. 이제 $a$와 $b$를 문자 $c$로 대체한 다음 이러한 과정을 반복하면 된다. 만약 자세한 사항이 궁금하다면 [여기](http://www.cs.utoronto.ca/~brudno/csc373w09/huffman.pdf)를 참고하자.


**Algorithm** Huffman($X$):<br>
&nbsp;&nbsp;&nbsp;&nbsp;**Input:** String $X$ of length $n$ with $d$ distinct characters<br>
&nbsp;&nbsp;&nbsp;&nbsp;**Output:** Coding tree for $X$<br>
&nbsp;&nbsp;&nbsp;&nbsp;Compute the frequency $f(c)$ of each character $c$ of $X$.<br>
&nbsp;&nbsp;&nbsp;&nbsp;Initialize a priority queue $Q$.<br>
&nbsp;&nbsp;&nbsp;&nbsp;**for each** character $c$ in $X$ **do**<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Create a single-node binary tree $T$ storing $c$.<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Insert $T$ into $Q$ with key $f(c)$.<br>
&nbsp;&nbsp;&nbsp;&nbsp;**while** $len(Q)$ > 1 **do**<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;($f_{1}$, $T_{1}$) = $Q$.remove_min()<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;($f_{2}$, $T_{2}$) = $Q$.remove_min()<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Create a new binary tree $T$ with left subtree $T_{1}$ and right subtree $T_{2}$<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Insert $T$ into $Q$ with key $f_{1} + f_{2}$<br>
&nbsp;&nbsp;&nbsp;&nbsp;($f$,$T$) = $Q$.remove_min()<br>
&nbsp;&nbsp;&nbsp;&nbsp;**return** tree $T$<br>

### 13.4.2 The Greedy Method
최적 인코딩을 만드는 허프만 알고리즘은 **greedy method**라 알려진 알고리즘 디자인 패턴의 응용 사례라 할 수 있다. 이 디자인 패턴은 특정한 구조의 일부 성질을 최소화하거나 최대화하면서 구조를 만드는 최적화 문제에서 주로 이용 된다. greedy method 패턴의 일반적인 공식은 brute-force 메소드 만큼이나 간단하다. greedy method를 이용해서 최적화 문제를 풀기 위해서는 우선 선택(choice)들의 시퀀스를 만들어야 한다. 이 시퀀스는 잘 이해할 수 있는 초기 조건부터 시작해서 그 초기 조건의 비용을 계산한다. 이제 이 패턴에서는 현재 가능한 모든 선택들의 비용을 비교하고, 가장 추가 비용이 적은 선택을 결정한다. 이러한 과정을 모든 선택에 대해 실시한다. 이 과정은 항상 최적해로 이어지지는 않는다.

그러나 이 메소드는 **greedy-choice** 성질을 갖는다고 알려진 문제들에 대해서 잘 작동한다. 만약 어떤 문제의 전역 최적해(global optimal solution)가 국소 최적해(현재 가능한 선택 중에서 가장 최선의 선택)를 연속적으로 찾는 과정을 통해 도출될 수 있다면 그 문제는 greedy-choice 성질을 갖는다고 한다.

## 13.5 Tries
이번 섹션에서는 텍스트를 전처리하는 문자열 탐색 알고리즘을 다룰 것이다. 이는 전처리를 하는 데 시간을 많이 쓰더라도 이후 쿼리에서 시간을 절약하고자 하는 상황에서 유용하다. 예를 들어 셰익스피어의 `Hamlet`에 대한 패턴 매칭을 제공하거나, `Hamlet`을 주제로 하는 웹 페이지를 제공하는 검색 엔진에서 이를 이용할 것이다.

**트라이(trie)**는 빠른 패턴 매칭을 지원하기 위해 문자열을 저장하는 트리 기반 자료 구조이다. 트라이의 주된 활용은 정보 검색(information retrieval)에 있는데, 사실 "trie"라는 이름도 "re*trie*val"에서 나온 것이다. 트라이가 지원하는 주된 쿼리 연산으로는 패턴 매칭과 **prefix** 매칭이 있다. prefix 매칭은 문자열 $X$가 주어졌을 때 $S$안에서 $X$를 prefix로 하는 모든 문자열을 찾는 것을 의미한다.

### 13.5.1 Standard Tries
알파벳 $\Sigma$로 이루어진 문자열 $s$들의 집합을 $S$라 하고, $S$ 안의 어떤 문자열도 다른 문자열의 prefix가 되지 않는다고 하자. $S$의 **표준 트라이(standard trie)**는 다음의 성질을 만족하는 순서 트리 $T$이다.
- 루트를 제외한 각각의 노드는 $\Sigma$의 문자로 label된다.
- 내부 노드의 자식들은 서로 다른 label을 갖는다.
- $T$는 $s$개의 잎을 갖고, 각각의 잎은 $S$의 문자열과 연결되어 있는데, 루트부터 잎 $v$까지의 경로에 있는 노드들의 label을 합치면 $v$와 연결된 $S$의 문자열이 된다.

즉, 트라이 $T$는 루트부터 잎까지의 경로를 통해 $S$의 문자열을 표현한다. 이 때 중요한 것은 $S$의 어떠한 문자열도 다른 문자열의 prefix가 되지 않으므로 $S$의 각각의 문자열이 $T$의 잎과 일대일로 매칭될 수 있다(이는 허프만 코딩에서 어떠한 code-word도 다른 문자열의 code-word가 되지 않는다고 한 것과 비슷하다). 이러한 가정이 비현실적이라 생각할 수도 있지만, 각각의 string 뒤에 알파벳 $\Sigma$에 속하지 않는 특별한 문자를 붙이면 이러한 가정을 언제나 만족하는 것이 가능하다.

<img width="640" alt="figure-13.10" src="https://user-images.githubusercontent.com/20944657/37556884-9e075926-2a3f-11e8-8b51-5aa34b6a2f0a.png">

표준 트라이 $T$의 내부 노드는 어디서나 1부터 $|\Sigma|$ 개의 자식을 갖는다. 루트부터 시작해서 깊이 $k$의 내부 노드 $v$까지의 경로는 $k$-character prefix $X[0:k]$와 대응된다. 사실 prefix $X[0:k]$ 뒤에 이어지는 모든 문자 $c$에 대해 $c$ label을 갖는 $v$의 자식이 존재한다. 이러한 방식을 통해 트라이는 문자열의 집합 안에 존재하는 공통적인 prefix를 정확하게 저장한다.

특별한 경우로, 만약 알파벳에 오직 문자가 두개만 존재하는 경우에는 일부 내부 노드만 하나의 자식을 갖는 이진 트리가 된다(improper binary tree). 일반적으로, 각각의 내부 노드는 $|\Sigma|$까지의 자식을 가질 수 있긴 하지만 실제로는 훨씬 더 적은 수의 자식을 갖는다. 예를 들어 위의 그림에서는 오직 하나의 자식만 갖는 여러 내부 노드가 있다. 큰 데이터 셋에서는 평균적으로 깊이가 깊어질수록 더 적은 수의 노드를 갖게 되는데, 이는 공통적인 prefix를 갖는 문자열이 적어지기 때문이다. 또, 많은 언어에서는 자연적으로 발생하기 힘든 문자 조합이 꽤나 많이 존재한다.

**Proposition 13.6:** 알파벳 $\Sigma$로 이루어져 있고 총 길이가 $n$인, $S$의 $s$ 문자열을 저장하는 표준 트라이는 다음의 성질을 만족한다:

- $T$의 높이는 $S$의 가장 긴 문자열의 길이와 같다.
- $T$의 모든 내부 노드는 최대 $|\Sigma|$개의 자식을 갖는다.
- $T$는 $s$개의 잎을 갖는다.
- $T$의 노드의 수는 최대 $n+1$이다.

트라이의 노드의 수에 있어서 worst case는 두 문자열이 어떠한 공통 prefix도 같지 않는 경우이다. 이러한 경우 모든 내부 노드는 하나의 자식만을 갖는다.

$S$의 트라이 $T$는 $S$의 문자열을 key로 갖는 set이나 map을 구현하는 데 이용할 수 있다. 길이가 $m$인 문자열의 탐색은 $O(m\cdot |\Sigma|)$가 걸리는데, 이는 $m+1$개의 노드를 방문하고 각각의 노드에서 자식이 그 다음 문자를 갖는 지 찾는 과정에서 $O(|\Sigma|)$ 시간을 쓰기 때문이다. 그런데 각각의 노드에서 자식들로의 매핑을 보조 탐색 테이블이나 해쉬 테이블을 이용해서 구현하거나, $|\Sigma|$가 충분히 작은 경우 각각의 노드에 대한 direct lookup table을 만든다면 노드에서의 시간을 $O(\log|\Sigma|)$나 expected $O(1)$으로 줄이는 것이 가능하다. 따라서 우리는 일반적으로 길이가 $m$인 문자열의 탐색에 $O(m)$ 시간이 걸린다고 예상한다.

위의 논의를 통해 우리는 트라이를 이용해서 특별한 패턴 매칭인 **단어 매칭(word matching)**을 할 수 있음을 알 수 있다. 단어 매칭은 표준 패턴 매칭과 달리 그 substring을 찾는 것은 불가능하지만 특정한 단어가 string 내부에 있는 지 알 수 있다. 이를 위해서는 원래 문서의 모든 단어가 트라이에 추가되어야 한다. 이를 확장하면 prefix-matching 쿼리를 구현하는 것도 가능하다. 그러나 단어의 suffix라던가, 두 단어를 연결한 단어라던가 하는 임의의 패턴에 대한 탐색을 효율적으로 구현하는 것은 힘들다.

표준적인 트라이를 만들기 위해 우리는 한번에 하나의 문자열을 추가하는 incremental 알고리즘으 ㄹ이용한다. 길이 $m$인 $X$를 추가하는 과정의 worst-case performance는 $O(m\cdot |\Sigma|)$이고, 각각의 노드에 해쉬 테이블을 이용하면 expected $O(m)$이 된다. 따라서 $S$의 트라이를 만드는 데에는 $O(n)$시간이 걸리고, 이 때 $n$은 $S$ 문자열의 전체 길이를 의미한다.

<img width="640" alt="figure-13.11a" src="https://user-images.githubusercontent.com/20944657/37557215-c91adcfa-2a44-11e8-9129-5bcd49c8a656.png">
<img width="640" alt="figure-13.11b" src="https://user-images.githubusercontent.com/20944657/37557216-c9474074-2a44-11e8-9c08-6e17282ff6e7.png">

### 13.5.2 Compressed Tries
표준 트라이는 공간적으로 비효율적이기 때문에 **Patricia trie**라고도 알려진 **압축 트라이(compressed trie)**를 이용한다. 압축 트라이는 표준 트라이와 비슷하지만 각각의 내부 노드가 적어도 두 개의 자식을 갖게끔 해서 공간 복잡도를 줄인다는 장점이 있다.

$T$를 표준 트라이라 하고, $T$의 내부 노드 $v$가 하나의 자식만 갖고 루트가 아닌 경우 **불필요하다(redundant)**고 하자. 또, 만약 $k \geq 2$개 엣지의 chain가

$$ (v_{0}, v_{1})(v_{1},v_{2})\cdots(v_{k-1},v_{k})$$

다음의 두 성질을 만족한다면 이 체인을 **불필요하다**고 한다:

- $v_{i}$, where $i=1,...,k-1$이 불필요하다
- $v_{0}과 v_{k}$는 불필요하지 않다


우리는 불필요한 체인 $ (v_{0}, v_{1})(v_{1},v_{2})\cdots(v_{k-1},v_{k})$을 하나의 엣지 $(v_{0}, v_{k})$로 합쳐서 압축 트리를 만들 수 있다. 이 때 $v_{1},...,v_{k}$를 합치고 이를 $v_{k}$라고 한다.

<img width="640" alt="figure-13.12" src="https://user-images.githubusercontent.com/20944657/37557281-7a2005ca-2a45-11e8-83f7-87bd252a3ba4.png">

표준 트라이와 달리 압축 트라이가 갖는 장점은 노드의 수가 총 길이가 아닌 문자열의 수에 비례한다는 점이다.

**Proposition 13.7:** 크기가 $d$인 알파벳으로 이루어진 $S$의 $s$ 문자열을 저장하는 압축 트리는 다음의 성질을 만족한다:

- $T$의 모든 내부 노드는 적어도 두개, 최대 $d$개의 자식을 갖는다.
- $T$는 $s$개의 잎 노드를 갖는다.
- $T$의 노드의 수는 $O(s)$이다.

그런데, 결국 각각의 노드의 규모가 커지므로 압축 트라이가 갖는 이점이 크지 않다는 생각을 할 수 있다. 실제로도 그러한데, 압축 트리는 이미 primary structure에 저장된 문자열들에 대한 **보조(auxiliary)** 인덱스 구조로 이용될 때에 진정한 효율을 보이고, 실제로 모든 문자열을 저장해야 할 필요는 없다.

예를 들어 우리가 노드에 $X$ label을 직접 저장하지 않고, $X = S[i][j:k]$라면 세 정수 $(i, j:k)$를 저장한다고 해보자.

<img width="640" alt="figure-13.13a" src="https://user-images.githubusercontent.com/20944657/37557320-33e3a070-2a46-11e8-83f0-b80c5097871b.png">
<img width="640" alt="figure-13.13b" src="https://user-images.githubusercontent.com/20944657/37557321-3411d792-2a46-11e8-856d-eef623946c00.png">

그러면 트라이의 전체 공간 사용량은 $O(n)$이 아니라 $O(s)$가 된다. 물론 실제 문자열을 저장하기 위한 공간이 필요하긴 하지만 트라이의 공간 사용량을 줄일 수 있다는 장점이 있다. 

압축 트라이의 탐색 시간은 노드의 수가 더 많은 표준 트라이에 비해 특별히 빠르다고 할 수 없는데, 트라이의 경로를 탐색하면서 여러 문자로 이루어진 노드 안에서도 탐색을 실시해야 하기 때문이다.


### 13.5.3 Suffix Tries
트라이의 주된 활용 사례 중 하나는 $S$의 문자열이 모두 $X$의 suffix인 경우이다. 이러한 경우의 트라이를 **suffix trie**(**suffix tree**, **position tree**)라고 한다. 이러한 경우 트라이의 표현은 위의 압축 트라이보다 더 간략하게 이루어질 수 있는데, $X[j,k]$를 나타내는 $(j,k)$만 저장하면 되기 때문이다. 이러한 표현 방식을 compact representation이라 한다. 아래는 "minimize"를 suffix trie로 나타낸 예제이다.

<img width="640" alt="figure-13.14a" src="https://user-images.githubusercontent.com/20944657/37557363-b8a89306-2a46-11e8-8647-daa7c65b6b31.png">

<img width="640" alt="figure-13.14b" src="https://user-images.githubusercontent.com/20944657/37557362-b87af9be-2a46-11e8-9048-5a15c50fe1a0.png">

suffix trie의 가장 큰 장점은 표준 트라이에 여러 공간 압축 알고리즘을 이용한 것보다도 더 공간을 절약할 수 있다는 것이다. 길이가 $n$인 문자열 $X$의 suffix들의 총 길이는 

$$ 1 + 2 + \cdots + n = \dfrac{n(n+1)}{2}$$

이므로 $X$의 모든 suffix를 저장하기 위해서는 $O(n^{2})$ 공간이 필요하다. 그런데, compact representation을 이용하는 suffix trie는 이 문자열들을 $O(n)$ 공간만으로 표현하는 것이 가능하다.

**Proposition 13.8:** 길이가 $n$인 문자열 $X$에 대한 suffix trie $T$의 compact representation은 $O(n)$ 공간 복잡도를 갖는다.

### Construction
우리는 13.5.1과 비슷한 incremental 알고리즘을 이용해서 suffix trie를 만들 수 있다. 이러한 과정은 suffix의 총 길이가 $n$에 quadratic하기 때문에 $O(|\Sigma|n^{2})$ 시간이 걸린다. 그러나 (compact) suffix trie는 특별한 알고리즘을 이용해서 $O(n)$ 시간 안에 만드는 것이 가능하다. 아쉽게도 이 알고리즘은 복잡해서 이 책에서 다루지 않을 것이다.

### 13.5.4 Search Engine Indexing
World Wide Web은 거대한 규모의 텍스트 문서들을 포함하고 있다. 이 페이지들에 대한 정보는 **웹 크롤러(Web Crawler)**라 불리는 프로그램에 의해서 수집되는데, 이 정보들은 특별한 딕셔너리 데이터베이스에 저장된다. **검색 엔진(search engine)**은 유저들로 하여금 이 데이터베이스에서 관련 정보를 검색(retrieve)할 수 있게 해주는데, 이 검색 엔진들은 입력받은 키워드를 이용해서 관련된 페이지들을 찾아준다. 이번 섹션에서는 간략화된 버전의 검색 엔진을 알아볼 것이다.

### Inverted Files
검색 엔진에 의해 저장된 핵심적인 정보는 **inverted index** 혹은 **inverted file**이라 불리는 딕셔너리인데, 이 딕셔너리는 단어 $w$와 $w$를 포함하는 페이지들의 집합인 $L$에 대한 key-value pair $(w,L)$을 저장한다. 이 딕셔너리의 key는 **index term**이라 불리고, 많은 수의 고유 명사(proper noun)와 단어(vocabulary entry)로 이루어져 있다. 이 딕셔너리의 항목들은 **occurrence list**라 불리고, 가능한 한 많은 웹 페이지를 cover한다.

우리는 다음과 같은 항목들로 이루어진 자료 구조를 이용해서 inverted index를 효율적으로 구현할 수 있다:
1. 해당 용어의 occurence list를 저장하는 배열(순서는 없다)
2. index term의 집합에 대한 압축 트라이. 이 압축 트라이의 잎은 관련된 용어의 occurence list의 인덱스를 저장하고 있다.

occurrence list를 트라이 밖에 저장하는 것은 트라이 자료 구조를 내부 메모리 안에 맞게끔 충분히 작은 사이즈로 만들기 위해서이다. occurrence list는 크기가 크기 때문에 메모리가 아닌 디스크에 저장되어야 한다. 이러한 자료 구조를 이용하면 하나의 키워드에 대한 쿼리는 word matching 쿼리를 날리는 것과 같아진다(Section 13.5.1). 즉, 우리는 트라이에서 그 키워드를 찾아 연결된 occurrence list를 반환한다.

만약 여러 키워드가 주어지고 그 키워드를 **모두** 포함하는 페이지들을 반환해야 한다면 우리는 각각의 키워드에 대한 occurrence list를 찾아 그 교집합을 반환해야 한다. 교집합 연산을 빠르게 하기 위해서 각각의 occurrence list는 효율적인 set 연산을 지원하기 위해 address로 정렬된 시퀀스나 map으로 구현되어야 한다.

입력받은 키워드를 포함하는 페이지들의 리스트를 반환하는 단순한 업무 말고도, 검색 엔진은 페이지들을 그 관련성이 큰 순서대로 **랭크를 매기는(ranking)** 추가적인 서비스를 제공한다. 빠르고 정확한 랭크 알고리즘을 만드는 것은 컴퓨터 연구자들과 전자 상거래 회사들의 중요한 목표 중 하나이다.